# OpenAI Whisper STT - Reconnaissance Vocale par API

**Module :** 01-Audio-Foundation  
**Niveau :** Debutant  
**Technologies :** OpenAI Whisper API, GPT-4o-Transcribe  
**Duree estimee :** 35 minutes  

## Objectifs d'Apprentissage

- [ ] Generer un echantillon audio de test avec l'API TTS
- [ ] Transcrire de l'audio avec `client.audio.transcriptions.create`
- [ ] Comprendre les formats de reponse (json, text, srt, vtt, verbose_json)
- [ ] Obtenir des timestamps au niveau des mots
- [ ] Utiliser la detection automatique de langue
- [ ] Traduire de l'audio vers l'anglais avec l'endpoint translation
- [ ] Comparer Whisper-1 et GPT-4o-Transcribe

## Prerequis

- Environment Setup (module 00) complete
- Cle API OpenAI configuree (`OPENAI_API_KEY` dans `.env`)
- Notebook 01-1 (TTS) recommande mais pas obligatoire

**Navigation :** [Index](../README.md) | [<< Precedent](01-1-OpenAI-TTS-Intro.ipynb) | [Suivant >>](01-3-Basic-Audio-Operations.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres STT
stt_model = "whisper-1"            # "whisper-1" ou "gpt-4o-transcribe"
language = None                    # Code langue ISO 639-1 (None = auto-detection)
response_format = "verbose_json"   # "json", "text", "srt", "vtt", "verbose_json"

# Configuration
generate_test_audio = True         # Generer un fichier audio de test via TTS
test_translation = True            # Tester l'endpoint de traduction
compare_models = True              # Comparer whisper-1 et gpt-4o-transcribe
save_results = True                # Sauvegarder les resultats de transcription

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
import tempfile
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
from io import BytesIO
import logging

# Lecture audio dans Jupyter
from IPython.display import Audio, display, HTML

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import transcribe_openai, synthesize_openai
        print("Helpers audio importes")
    except ImportError:
        print("Helpers audio non disponibles - mode autonome")

# Repertoires
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'stt'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
SAMPLES_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'samples'
SAMPLES_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('openai_whisper')

print(f"OpenAI Whisper STT - Reconnaissance Vocale")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, Modele STT : {stt_model}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes
OpenAI Whisper STT - Reconnaissance Vocale
Date : 2026-02-18 01:00:30
Mode : interactive, Modele STT : whisper-1
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\stt


In [3]:
# Chargement de la configuration et validation API
from dotenv import load_dotenv

# Recherche du .env dans les parents
current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve dans l'arborescence")

# Verification cle API OpenAI
openai_key = os.getenv('OPENAI_API_KEY')

if not openai_key:
    if notebook_mode == "batch" and not generate_test_audio:
        print("Mode batch sans generation : cle API ignoree")
        openai_key = "dummy_key_for_validation"
    else:
        raise ValueError(
            "OPENAI_API_KEY manquante dans .env\n"
            "Obtenez votre cle sur : https://platform.openai.com/api-keys"
        )

# Initialisation client OpenAI
from openai import OpenAI

client = OpenAI(api_key=openai_key)

# Test de connexion
if openai_key != "dummy_key_for_validation":
    try:
        models = client.models.list()
        whisper_models = [m.id for m in models if 'whisper' in m.id or 'transcribe' in m.id]
        print(f"Connexion API reussie")
        print(f"Modeles STT detectes : {whisper_models}")
    except Exception as e:
        print(f"Erreur connexion : {str(e)[:100]}")

print(f"\nConfiguration STT :")
print(f"  Modele : {stt_model}")
print(f"  Langue : {language or 'auto-detection'}")
print(f"  Format reponse : {response_format}")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


Connexion API reussie
Modeles STT detectes : ['gpt-4o-transcribe', 'gpt-4o-mini-transcribe', 'gpt-4o-transcribe-diarize', 'gpt-4o-mini-transcribe-2025-12-15', 'gpt-4o-mini-transcribe-2025-03-20', 'whisper-1']

Configuration STT :
  Modele : whisper-1
  Langue : auto-detection
  Format reponse : verbose_json


## Section 1 : Generation d'un echantillon de test

Avant de transcrire, nous avons besoin d'un fichier audio. Nous utilisons l'API TTS pour generer un echantillon controle, ce qui nous permettra de verifier la precision de la transcription.

Cette approche "round-trip" (TTS puis STT) est utile pour :
- Valider le pipeline audio de bout en bout
- Mesurer la fidelite de la transcription
- Avoir un texte de reference pour comparaison

In [4]:
# Generation d'echantillons audio de test
print("GENERATION DES ECHANTILLONS DE TEST")
print("=" * 45)

# Textes de test dans differentes langues
test_texts = {
    "fr": (
        "La reconnaissance vocale permet de convertir la parole en texte. "
        "Cette technologie utilise des reseaux de neurones profonds pour "
        "analyser le signal audio et identifier les mots prononces."
    ),
    "en": (
        "Speech recognition converts spoken language into written text. "
        "Modern systems use deep neural networks trained on thousands "
        "of hours of audio data to achieve human-level accuracy."
    ),
    "multi": (
        "Bonjour, je parle francais. Now I switch to English. "
        "Et je reviens au francais pour terminer."
    )
}

audio_samples = {}

if generate_test_audio:
    for lang_key, text in test_texts.items():
        print(f"\nGeneration echantillon '{lang_key}'...")
        print(f"  Texte : {text[:80]}...")

        response = client.audio.speech.create(
            model="tts-1",
            voice="nova",
            input=text,
            response_format="mp3"
        )

        audio_data = response.content
        filepath = SAMPLES_DIR / f"sample_{lang_key}.mp3"
        with open(filepath, 'wb') as f:
            f.write(audio_data)

        audio_samples[lang_key] = {
            "path": filepath,
            "text_original": text,
            "size_kb": len(audio_data) / 1024
        }

        print(f"  Fichier : {filepath.name} ({len(audio_data)/1024:.1f} KB)")
        display(Audio(data=audio_data, autoplay=False))

    print(f"\n{len(audio_samples)} echantillons generes")
else:
    # Chercher des fichiers existants
    for lang_key in test_texts:
        filepath = SAMPLES_DIR / f"sample_{lang_key}.mp3"
        if filepath.exists():
            audio_samples[lang_key] = {
                "path": filepath,
                "text_original": test_texts[lang_key],
                "size_kb": filepath.stat().st_size / 1024
            }
    print(f"{len(audio_samples)} echantillons existants trouves")

GENERATION DES ECHANTILLONS DE TEST

Generation echantillon 'fr'...
  Texte : La reconnaissance vocale permet de convertir la parole en texte. Cette technolog...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Fichier : sample_fr.mp3 (230.2 KB)



Generation echantillon 'en'...
  Texte : Speech recognition converts spoken language into written text. Modern systems us...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Fichier : sample_en.mp3 (214.7 KB)



Generation echantillon 'multi'...
  Texte : Bonjour, je parle francais. Now I switch to English. Et je reviens au francais p...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Fichier : sample_multi.mp3 (105.9 KB)



3 echantillons generes


## Section 2 : Transcription avec Whisper API

L'API Whisper accepte un fichier audio et retourne la transcription. Les parametres principaux :

| Parametre | Description | Valeurs |
|-----------|-------------|--------|
| `model` | Modele STT | `whisper-1`, `gpt-4o-transcribe` |
| `file` | Fichier audio | MP3, MP4, WAV, FLAC, etc. (max 25 MB) |
| `language` | Langue source | Code ISO 639-1 (`fr`, `en`, etc.) ou None |
| `response_format` | Format de sortie | `json`, `text`, `srt`, `vtt`, `verbose_json` |
| `timestamp_granularities` | Precision timestamps | `["word"]`, `["segment"]`, `["word", "segment"]` |

In [5]:
# Transcription de base
print("TRANSCRIPTION WHISPER")
print("=" * 45)

if "fr" in audio_samples:
    sample = audio_samples["fr"]
    print(f"Fichier source : {sample['path'].name}")
    print(f"Texte original : {sample['text_original'][:80]}...")

    # --- Format JSON simple ---
    print(f"\n--- Transcription (format json) ---")
    with open(sample['path'], 'rb') as audio_file:
        transcript_json = client.audio.transcriptions.create(
            model=stt_model,
            file=audio_file,
            response_format="json"
        )
    print(f"Resultat : {transcript_json.text}")

    # --- Format verbose_json (avec metadonnees) ---
    print(f"\n--- Transcription (format verbose_json) ---")
    with open(sample['path'], 'rb') as audio_file:
        transcript_verbose = client.audio.transcriptions.create(
            model=stt_model,
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["word", "segment"]
        )

    print(f"Texte : {transcript_verbose.text}")
    print(f"Langue detectee : {transcript_verbose.language}")
    print(f"Duree : {transcript_verbose.duration:.1f}s")

    # Affichage des segments
    if hasattr(transcript_verbose, 'segments') and transcript_verbose.segments:
        print(f"\nSegments ({len(transcript_verbose.segments)}) :")
        for seg in transcript_verbose.segments:
            print(f"  [{seg.start:.1f}s - {seg.end:.1f}s] {seg.text.strip()}")

    # Affichage des mots avec timestamps
    if hasattr(transcript_verbose, 'words') and transcript_verbose.words:
        print(f"\nMots avec timestamps ({len(transcript_verbose.words)}) :")
        for word in transcript_verbose.words[:15]:  # Premiers 15 mots
            print(f"  [{word.start:.2f}s - {word.end:.2f}s] {word.word}")
        if len(transcript_verbose.words) > 15:
            print(f"  ... ({len(transcript_verbose.words) - 15} mots supplementaires)")

    # Sauvegarde
    if save_results:
        result_file = OUTPUT_DIR / "transcription_fr.json"
        with open(result_file, 'w', encoding='utf-8') as f:
            json.dump({
                "text": transcript_verbose.text,
                "language": transcript_verbose.language,
                "duration": transcript_verbose.duration,
                "model": stt_model
            }, f, indent=2, ensure_ascii=False)
        print(f"\nResultat sauvegarde : {result_file.name}")
else:
    print("Aucun echantillon audio disponible")

TRANSCRIPTION WHISPER
Fichier source : sample_fr.mp3
Texte original : La reconnaissance vocale permet de convertir la parole en texte. Cette technolog...

--- Transcription (format json) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


Resultat : La reconnaissance vocale permet de convertir la parole en texte. Cette technologie utilise des réseaux de neurones profonds pour analyser le signal audio et identifier les mots prononces.

--- Transcription (format verbose_json) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


Texte : La reconnaissance vocale permet de convertir la parole en texte. Cette technologie utilise des réseaux de neurones profonds pour analyser le signal audio et identifier les mots prononces.
Langue detectee : french
Duree : 11.8s

Segments (3) :
  [0.0s - 3.9s] La reconnaissance vocale permet de convertir la parole en texte.
  [4.4s - 7.1s] Cette technologie utilise des réseaux de neurones profonds
  [7.1s - 10.7s] pour analyser le signal audio et identifier les mots prononces.

Mots avec timestamps (28) :
  [0.00s - 0.14s] La
  [0.14s - 0.82s] reconnaissance
  [0.82s - 1.24s] vocale
  [1.24s - 1.60s] permet
  [1.60s - 1.88s] de
  [1.88s - 2.30s] convertir
  [2.30s - 2.48s] la
  [2.48s - 2.74s] parole
  [2.74s - 3.00s] en
  [3.00s - 3.90s] texte
  [4.44s - 4.52s] Cette
  [4.52s - 5.12s] technologie
  [5.12s - 5.60s] utilise
  [5.60s - 5.80s] des
  [5.80s - 6.04s] réseaux
  ... (13 mots supplementaires)

Resultat sauvegarde : transcription_fr.json


### Interpretation : Transcription Whisper

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Fidelite | Tres elevee | Whisper reproduit le texte original avec precision |
| Detection de langue | Automatique | Whisper detecte la langue sans parametre explicite |
| Timestamps | Niveau mot et segment | Utile pour sous-titrage, karaoke, synchronisation |

**Points cles** :
1. Le format `verbose_json` fournit les metadonnees les plus completes
2. Les timestamps au niveau mot necessitent `timestamp_granularities=["word"]`
3. La detection de langue est fiable pour les langues courantes

## Section 3 : Formats de sortie specialises

Whisper supporte des formats standards pour le sous-titrage :

| Format | Description | Usage |
|--------|-------------|-------|
| `srt` | SubRip Subtitle | Lecteurs video, YouTube |
| `vtt` | WebVTT | Navigateurs web, HTML5 video |
| `text` | Texte brut | Traitement de texte, NLP |

In [6]:
# Formats de sous-titrage
print("FORMATS DE SOUS-TITRAGE")
print("=" * 45)

if "fr" in audio_samples:
    sample = audio_samples["fr"]
    subtitle_formats = ["text", "srt", "vtt"]

    for fmt in subtitle_formats:
        print(f"\n--- Format : {fmt.upper()} ---")
        with open(sample['path'], 'rb') as audio_file:
            result = client.audio.transcriptions.create(
                model=stt_model,
                file=audio_file,
                response_format=fmt
            )

        # Le resultat est une chaine pour text/srt/vtt
        output_text = result if isinstance(result, str) else str(result)
        print(output_text[:300])

        # Sauvegarde
        if save_results:
            ext = fmt
            result_file = OUTPUT_DIR / f"transcription_fr.{ext}"
            with open(result_file, 'w', encoding='utf-8') as f:
                f.write(output_text)
            print(f"Sauvegarde : {result_file.name}")

    print(f"\nTous les formats generes avec succes")
else:
    print("Aucun echantillon audio disponible")

FORMATS DE SOUS-TITRAGE

--- Format : TEXT ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


La reconnaissance vocale permet de convertir la parole en texte. Cette technologie utilise des réseaux de neurones profonds pour analyser le signal audio et identifier les mots prononces.

Sauvegarde : transcription_fr.text

--- Format : SRT ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


1
00:00:00,000 --> 00:00:03,500
La reconnaissance vocale permet de convertir la parole en texte.

2
00:00:04,260 --> 00:00:07,260
Cette technologie utilise des réseaux de neurones profonds

3
00:00:07,260 --> 00:00:10,760
pour analyser le signal audio et identifier les mots prononces.



Sauvegarde : transcription_fr.srt

--- Format : VTT ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


WEBVTT

00:00:00.000 --> 00:00:03.500
La reconnaissance vocale permet de convertir la parole en texte.

00:00:04.260 --> 00:00:07.260
Cette technologie utilise des réseaux de neurones profonds

00:00:07.260 --> 00:00:10.760
pour analyser le signal audio et identifier les mots prononces.


Sauvegarde : transcription_fr.vtt

Tous les formats generes avec succes


## Section 4 : Traduction et detection multilingue

L'API Whisper offre un endpoint de traduction (`translations`) qui traduit l'audio en anglais, quelle que soit la langue source.

| Endpoint | Entree | Sortie |
|----------|--------|--------|
| `transcriptions` | Audio (n'importe quelle langue) | Texte dans la langue source |
| `translations` | Audio (n'importe quelle langue) | Texte en anglais |

In [7]:
# Traduction et detection multilingue
print("TRADUCTION ET DETECTION MULTILINGUE")
print("=" * 45)

if test_translation and "fr" in audio_samples:
    sample_fr = audio_samples["fr"]

    # --- Transcription (langue source) ---
    print("\n--- Transcription (francais -> francais) ---")
    with open(sample_fr['path'], 'rb') as audio_file:
        transcript = client.audio.transcriptions.create(
            model=stt_model,
            file=audio_file,
            response_format="json"
        )
    print(f"Resultat : {transcript.text}")

    # --- Traduction (langue source -> anglais) ---
    print("\n--- Traduction (francais -> anglais) ---")
    with open(sample_fr['path'], 'rb') as audio_file:
        translation = client.audio.translations.create(
            model=stt_model,
            file=audio_file,
            response_format="json"
        )
    print(f"Resultat : {translation.text}")

    # --- Test multilingue ---
    if "multi" in audio_samples:
        print("\n--- Detection multilingue ---")
        sample_multi = audio_samples["multi"]
        print(f"Texte original : {sample_multi['text_original']}")

        with open(sample_multi['path'], 'rb') as audio_file:
            transcript_multi = client.audio.transcriptions.create(
                model=stt_model,
                file=audio_file,
                response_format="verbose_json"
            )
        print(f"Transcription : {transcript_multi.text}")
        print(f"Langue detectee : {transcript_multi.language}")

    print(f"\nTraduction et detection terminees")
else:
    print("Traduction desactivee ou echantillons manquants")

TRADUCTION ET DETECTION MULTILINGUE

--- Transcription (francais -> francais) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


Resultat : La reconnaissance vocale permet de convertir la parole en texte. Cette technologie utilise des réseaux de neurones profonds pour analyser le signal audio et identifier les mots prononces.

--- Traduction (francais -> anglais) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/translations "HTTP/1.1 200 OK"


Resultat : Vocal recognition converts speech into text. This technology uses deep neural networks to analyze audio signals and identify pronounced words.

--- Detection multilingue ---
Texte original : Bonjour, je parle francais. Now I switch to English. Et je reviens au francais pour terminer.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


Transcription : Bonjour, je parle francais. Et je reviens au francais pour terminer.
Langue detectee : french

Traduction et detection terminees


### Interpretation : Traduction et multilingue

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Traduction | Francais vers anglais | Whisper traduit directement sans etape intermediaire |
| Multilingue | Detection langue dominante | Whisper identifie la langue principale du fichier |
| Code-switching | Gere partiellement | Le melange de langues peut reduire la precision |

> **Note technique** : L'endpoint `translations` ne supporte que la traduction vers l'anglais. Pour d'autres langues cibles, combiner Whisper STT + un LLM de traduction.

## Section 5 : Comparaison Whisper-1 vs GPT-4o-Transcribe

OpenAI propose deux modeles de transcription :

| Modele | Architecture | Forces | Limitations |
|--------|-------------|--------|-------------|
| `whisper-1` | Whisper V2 | Rapide, stable, multilingue | Precision moderee sur accents forts |
| `gpt-4o-transcribe` | GPT-4o | Meilleure comprehension contextuelle | Plus lent, plus cher |

In [8]:
# Comparaison des modeles STT
print("COMPARAISON MODELES STT")
print("=" * 45)

stt_models_to_test = ["whisper-1"]
if compare_models:
    stt_models_to_test.append("gpt-4o-transcribe")

comparison_results = {}

if "fr" in audio_samples:
    sample = audio_samples["fr"]

    for model in stt_models_to_test:
        print(f"\n--- Modele : {model} ---")
        start_time = time.time()

        try:
            with open(sample['path'], 'rb') as audio_file:
                transcript = client.audio.transcriptions.create(
                    model=model,
                    file=audio_file,
                    response_format="verbose_json"
                )

            elapsed = time.time() - start_time

            comparison_results[model] = {
                "text": transcript.text,
                "time": elapsed,
                "language": transcript.language,
                "duration": transcript.duration
            }

            print(f"  Texte : {transcript.text}")
            print(f"  Langue : {transcript.language}")
            print(f"  Duree audio : {transcript.duration:.1f}s")
            print(f"  Temps API : {elapsed:.2f}s")

        except Exception as e:
            print(f"  Erreur : {str(e)[:100]}")
            comparison_results[model] = {"error": str(e)}

    # Tableau comparatif
    if len(comparison_results) > 1:
        print(f"\nTableau comparatif :")
        print(f"{'Modele':<25} {'Temps API':<12} {'Langue':<10}")
        print("-" * 47)
        for model, data in comparison_results.items():
            if "error" not in data:
                print(f"{model:<25} {data['time']:<12.2f} {data['language']:<10}")
            else:
                print(f"{model:<25} {'ERREUR':<12} {'-':<10}")
else:
    print("Aucun echantillon audio disponible")

COMPARAISON MODELES STT

--- Modele : whisper-1 ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


  Texte : La reconnaissance vocale permet de convertir la parole en texte. Cette technologie utilise des réseaux de neurones profonds pour analyser le signal audio et identifier les mots prononces.
  Langue : french
  Duree audio : 11.8s
  Temps API : 1.47s

--- Modele : gpt-4o-transcribe ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 400 Bad Request"


  Erreur : Error code: 400 - {'error': {'message': "response_format 'verbose_json' is not compatible with model

Tableau comparatif :
Modele                    Temps API    Langue    
-----------------------------------------------
whisper-1                 1.47         french    
gpt-4o-transcribe         ERREUR       -         


In [9]:
# Mode interactif
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF")
    print("=" * 50)
    print("\nEntrez un texte a synthetiser puis transcrire (round-trip) :")
    print("(Laissez vide pour passer a la suite)")

    try:
        user_text = input("\nVotre texte : ")

        if user_text.strip():
            # Generation TTS
            print(f"\n1. Generation TTS...")
            tts_response = client.audio.speech.create(
                model="tts-1",
                voice="nova",
                input=user_text,
                response_format="mp3"
            )
            print(f"   Audio genere ({len(tts_response.content)/1024:.1f} KB)")
            display(Audio(data=tts_response.content, autoplay=False))

            # Sauvegarde temporaire pour transcription
            temp_path = OUTPUT_DIR / "interactive_sample.mp3"
            with open(temp_path, 'wb') as f:
                f.write(tts_response.content)

            # Transcription
            print(f"\n2. Transcription STT...")
            with open(temp_path, 'rb') as audio_file:
                transcript = client.audio.transcriptions.create(
                    model=stt_model,
                    file=audio_file,
                    response_format="json"
                )

            print(f"   Original    : {user_text}")
            print(f"   Transcrit   : {transcript.text}")

            # Comparaison
            match = user_text.lower().strip() == transcript.text.lower().strip()
            print(f"   Correspondance exacte : {'Oui' if match else 'Non'}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

MODE INTERACTIF

Entrez un texte a synthetiser puis transcrire (round-trip) :
(Laissez vide pour passer a la suite)
Mode interactif non disponible (execution automatisee)


## Bonnes pratiques et analyse des couts

### Optimisation de la transcription

| Strategie | Description | Impact |
|-----------|-------------|--------|
| Specifier la langue | `language="fr"` | Precision accrue, latence reduite |
| Audio propre | Reduire le bruit de fond | Meilleure qualite de transcription |
| Segmenter les longs fichiers | Decouper en segments < 25 MB | Evite les timeouts |
| Utiliser verbose_json | Exploiter les timestamps | Utile pour le sous-titrage |

### Grille tarifaire (Janvier 2025)

| Modele | Cout | Equivalent |
|--------|------|------------|
| `whisper-1` | $0.006 / minute | ~$0.36 / heure |
| `gpt-4o-transcribe` | $0.006 / minute | ~$0.36 / heure |

In [10]:
# Statistiques de session et prochaines etapes
print("STATISTIQUES DE SESSION")
print("=" * 45)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele STT : {stt_model}")
print(f"Format reponse : {response_format}")
print(f"Echantillons generes : {len(audio_samples)}")

if comparison_results:
    print(f"Modeles compares : {list(comparison_results.keys())}")

if save_results:
    saved = list(OUTPUT_DIR.glob('*'))
    print(f"Fichiers sauvegardes : {len(saved)} dans {OUTPUT_DIR}")

print(f"\nPROCHAINES ETAPES")
print(f"1. Decouvrir les operations audio de base (01-3-Basic-Audio-Operations)")
print(f"2. Tester Whisper en local avec GPU (01-4-Whisper-Local)")
print(f"3. Essayer le TTS local avec Kokoro (01-5-Kokoro-TTS-Local)")
print(f"4. Explorer la comparaison multi-modeles (03-1)")

print(f"\nNotebook Whisper STT termine - {datetime.now().strftime('%H:%M:%S')}")

STATISTIQUES DE SESSION
Date : 2026-02-18 01:00:56
Modele STT : whisper-1
Format reponse : verbose_json
Echantillons generes : 3
Modeles compares : ['whisper-1', 'gpt-4o-transcribe']
Fichiers sauvegardes : 4 dans D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\stt

PROCHAINES ETAPES
1. Decouvrir les operations audio de base (01-3-Basic-Audio-Operations)
2. Tester Whisper en local avec GPU (01-4-Whisper-Local)
3. Essayer le TTS local avec Kokoro (01-5-Kokoro-TTS-Local)
4. Explorer la comparaison multi-modeles (03-1)

Notebook Whisper STT termine - 01:00:56


---

# CHALLENGE BONUS - Sous-titrage Automatique

**Points : 0.5 pts**

## Objectif

Créer des **sous-titres synchronisés** pour un court extrait audio.

## Ce que vous avez appris

Ce notebook montre:
- **Section 2** : Transcription avec timestamps (`verbose_json`)
- **Section 3** : Formats de sous-titrage (SRT, VTT)
- **Section 4** : Traduction audio vers anglais

## Criteres de succes

- [ ] Générer un audio de test avec TTS (15-20 secondes de texte)
- [ ] Transcrire avec `response_format="verbose_json"` et `timestamp_granularities=["word"]`
- [ ] Formater manuellement les sous-titres en **SRT** (avec numérotation)
- [ ] Chaque sous-titre doit durer **max 5 secondes**
- [ ] Sauvegarder le fichier .srt

## Contraintes techniques

- Utiliser les timestamps de `transcript.words`
- Regrouper les mots en segments de 5-8 mots maximum
- Format SRT standard avec numérotation

---

**Soumission** : PR avec titre "Challenge #9 - [Votre Nom]", fichier SRT et extrait audio